# RSI.csv Generator RoadCare

In [1]:
import os
import glob
import torch
import pickle
import numpy as np
import pandas as pd
from scipy.stats import mode
from math import cos,asin,sqrt,pi
from torch.autograd import Variable
from libraries.road_care_nn import ROAD_INDEXING_CNN_NET_2

# Load Model

In [2]:
def load_model_and_scaler():
    model = ROAD_INDEXING_CNN_NET_2(832,1024,1024,256,128,3)
    model.load_state_dict(torch.load("./libraries/our_cnn_model_lr_0.0005_bl_ROADCARE.pth",
                                     map_location=torch.device('cpu')))
    model.eval()

    with open('./libraries/scaler.pickle', 'rb') as fp:
        scaler = pickle.load(fp)

    return model,scaler

def predict(model,scaler,X):
    if X.shape[0]==0:
        return np.array([1])
    X=scaler.transform(X)
    X = torch.Tensor(X[:,np.newaxis,:]).to("cpu:0")
    output = model(Variable(X))
    _ , predicted = torch.max(output.data,1)
    return predicted.numpy()

model,scaler=load_model_and_scaler()

# RoadCare Processing

In [3]:
def orient_axis(x,y,z):
    g=9.8
    if(abs(abs(z)-g)<abs(abs(y)-g) and abs(abs(z)-g)<abs(abs(x)-g)):
        #print("Z Normal")
        return [x,y,z]
    elif(abs(abs(x)-g)<abs(abs(z)-g) and abs(abs(x)-g)<abs(abs(y)-g)):
        #print("X=Z")
        return [z,y,x]
    elif(abs(abs(y)-g)<abs(abs(z)-g) and abs(abs(y)-g)<abs(abs(x)-g)):
        #print("Y=Z")
        return [x,z,y]
    else:
        return [x,y,z]
    
def convert_to_50hz(b):
    return b.reset_index(drop=True).iloc[np.linspace(0,b.shape[0]-1,50,dtype="int32"),:].reset_index(drop=True)

def read_acc_file(acc_file):
    df=pd.read_csv(acc_file)
    df.columns=['acc_x','acc_y','acc_z','time','mark']
    df_filtered=df[['acc_x','acc_y','acc_z','time']].copy()

    #Convert frequency
    df_data=df_filtered.groupby('time').apply(lambda e:convert_to_50hz(e)).reset_index(drop=True)

    #Orient the axis
    df_data[['acc_x','acc_y','acc_z']]=\
    np.array(df_data[['acc_x','acc_y','acc_z']].apply(lambda e:orient_axis(e[0],e[1],e[2]),axis=1).to_list())

    return df_data

def read_gps_file(gps_file):
    df_gps=pd.read_csv(gps_file)
    df_gps.columns=['lat','long','speed','altitude','time','mark']
    df_gps_filter=df_gps[['lat','long','speed','time']].copy()
    return df_gps_filter

def process_acc_data(acc_data):
    cols=\
    ['time']+\
    [f'acc_{i}' for i in range(3*150)]+\
    [f'acc_mean_{i}' for i in range(3)]+\
    [f'acc_median_{i}' for i in range(3)]+\
    [f'acc_min_{i}' for i in range(3)]+\
    [f'acc_max_{i}' for i in range(3)]+\
    [f'acc_std_{i}' for i in range(3)]+\
    [f'acc_rms_{i}' for i in range(3)]

    series=[]
    last_index=acc_data.shape[0]-100
    for i in range(0,last_index,50):
        temp=acc_data.iloc[i:i+150]
        series.append(
        #raw values are like [150 acc_z...150acc_y,150 acc_x] now its not like that see .T is removed 150*(acc_x,acc_y,acc_z)
        [temp.time.iloc[-1]]+\
        temp[['acc_x','acc_y','acc_z']].values.flatten().tolist()+\
        temp[['acc_x','acc_y','acc_z']].mean().values.tolist()+\
        temp[['acc_x','acc_y','acc_z']].median().values.tolist()+\
        temp[['acc_x','acc_y','acc_z']].min().values.tolist()+\
        temp[['acc_x','acc_y','acc_z']].max().values.tolist()+\
        temp[['acc_x','acc_y','acc_z']].std().values.tolist()+\
        temp[['acc_x','acc_y','acc_z']].pow(2).mean().pow(0.5).values.tolist()
        )
    return pd.DataFrame(np.array(series),columns=cols)

def process_speed_data(gps_data):
    cols=\
    ['time']+\
    [f'speed_{i}' for i in range(3)]+\
    [f's_mean_{i}' for i in range(1)]+\
    [f's_median_{i}' for i in range(1)]+\
    [f's_min_{i}' for i in range(1)]+\
    [f's_max_{i}' for i in range(1)]+\
    [f's_std_{i}' for i in range(1)]+\
    [f's_rms_{i}' for i in range(1)]

    series=[]
    last_index=gps_data.shape[0]-2
    for i in range(0,last_index,1):
        temp=gps_data.iloc[i:i+3]
        series.append(
            [temp.time.iloc[-1]]+\
            temp[['speed']].values.flatten().tolist()+\
            temp[['speed']].mean().values.tolist()+\
            temp[['speed']].median().values.tolist()+\
            temp[['speed']].min().values.tolist()+\
            temp[['speed']].max().values.tolist()+\
            temp[['speed']].std().values.tolist()+\
            temp[['speed']].pow(2).mean().pow(0.5).values.tolist()
            )
    return pd.DataFrame(np.array(series),columns=cols)

def process_roadCare_features(folder):
    acc_file=folder+"ACC.csv"
    gps_file=folder+"GPS.csv"

    df_acc=read_acc_file(acc_file)
    df_gps=read_gps_file(gps_file)

    acc_processed=process_acc_data(df_acc)
    gps_processed=process_speed_data(df_gps[['speed','time']])

    merged=pd.merge(left=acc_processed,right=gps_processed,how= 'inner',on='time')
    return merged,df_gps #merged and gps_df for vel

# Helpers

In [4]:
def distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742000 * asin(sqrt(a))

def get_time_in_distance(stop_lat,stop_long,stop_time,vel_df,d):
    for lat,long,time in vel_df[['lat','long','time']].values:
        #print(lat,long,time)
        if distance(stop_lat,stop_long,lat,long)<=d:
            return time,stop_time
        
def get_slice(merged,start_time,stop_time):
    if start_time<=stop_time:
        df_sliced=merged[(merged.time>=start_time)&(merged.time<=stop_time)].copy()
    else:
        df_sliced=merged[(merged.time>=stop_time)&(merged.time<=start_time)].copy()
    return df_sliced.drop(columns=['time'])

def get_majority_r_class(model,scaler,merged,stop_lat,stop_long,stop_time,vel_df,d):
    #'07/01/2019 09:05:51','07/01/2019 09:05:59'
    start_time,stop_time=get_time_in_distance(stop_lat,stop_long,stop_time,vel_df,d)
    sliced=get_slice(merged,start_time,stop_time)
    return mode(predict(model,scaler,sliced)).mode[0]

# Processing RSI

In [5]:
folders=glob.glob('./Trails/*/*/')
road_distance=50

In [6]:
for folder in folders:
    if os.path.isfile(folder+'RSI_rc.csv'): #checking weither RSI_rc file exist already?
        print(f'folder: {folder} is already processed and Augumented\n') #file is already processed
    else:
        merged,vel_df=process_roadCare_features(folder)
        ana_df=pd.read_csv(folder+'Analysis_GPS.csv')

        ana_df['R_class']=\
        ana_df[['lat',
                'long',
                'start_date',
                'start_time']].apply(lambda t:get_majority_r_class(model,scaler,merged,t[0],t[1],t[2]+" "+t[3],
                                                                   vel_df,road_distance),
                                     axis=1)

        ana_df[['R_class']].to_csv(folder+'RSI_rc.csv',index=False)
        print(f'Processed and added RSI to file "RSI_rc.csv" in folder: {folder}\n')

folder: ./Trails\down\01-07-2019\ is already processed and Augumented

folder: ./Trails\down\01-07-2019_DATA_17_03_44\ is already processed and Augumented

folder: ./Trails\down\02-07-2019\ is already processed and Augumented

folder: ./Trails\down\02-07-2019_DATA_09_52_39\ is already processed and Augumented

folder: ./Trails\down\02-07-2019_DATA_11_11_23\ is already processed and Augumented

folder: ./Trails\down\02-07-2019_DATA_12_06_05\ is already processed and Augumented

folder: ./Trails\down\03-07-2019\ is already processed and Augumented

folder: ./Trails\down\03-07-2019_DATA_09_56_40\ is already processed and Augumented

folder: ./Trails\down\03-07-2019_DATA_11_52_11\ is already processed and Augumented

folder: ./Trails\down\03-07-2019_DATA_18_04_00\ is already processed and Augumented

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\down\03-09-2019\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\down\03-09-2019_DATA_18_38_56\

Processed an

C:\Users\WER\AppData\Local\Temp/ipykernel_7012/1945594337.py:97: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  df_acc=read_acc_file(acc_file)


Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\down\04-09-2019_DATA_18_29_36\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\down\05-07-2019\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\down\05-07-2019_DATA_09_21_15\



C:\Users\WER\AppData\Local\Temp/ipykernel_7012/1945594337.py:97: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  df_acc=read_acc_file(acc_file)


Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\down\05-07-2019_DATA_10_56_28\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\down\05-07-2019_DATA_11_49_44\



...

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\down\05-07-2019_DATA_17_01_55\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\down\05-07-2019_DATA_18_56_42\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\down\05-09-2019\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\down\06-07-2019\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\down\06-07-2019_DATA_09_23_17\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\down\06-07-2019_DATA_09_48_06\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\down\06-07-2019_DATA_10_58_09\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\down\06-07-2019_DATA_11_51_02\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\down\06-07-2019_DATA_18_03_58\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\down\12-09-2019\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\down\13

C:\Users\WER\AppData\Local\Temp/ipykernel_7012/1945594337.py:97: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  df_acc=read_acc_file(acc_file)


Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\up\04-09-2019_DATA_17_21_38\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\up\05-07-2019\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\up\05-07-2019_DATA_08_28_46\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\up\05-07-2019_DATA_10_09_30\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\up\05-07-2019_DATA_10_46_37\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\up\05-07-2019_DATA_16_16_42\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\up\05-07-2019_DATA_18_03_49\



C:\Users\WER\AppData\Local\Temp/ipykernel_7012/1945594337.py:97: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  df_acc=read_acc_file(acc_file)


Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\up\05-09-2019\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\up\06-07-2019\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\up\06-07-2019_DATA_08_40_03\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\up\06-07-2019_DATA_08_57_09\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\up\06-07-2019_DATA_10_07_40\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\up\06-07-2019_DATA_11_00_08\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\up\06-07-2019_DATA_16_58_00\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\up\12-09-2019\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\up\13-09-2019\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\up\13-09-2019_DATA_12_58_59\

Processed and added RSI to file "RSI_rc.csv" in folder: ./Trails\up\13-09-2019_DATA_16_29_45\

Processed a

In [7]:
#NICE
#mode road class in last 50 meter road segment